# Run a NN with x and $\xi$ as the input to the NN model and second stage objective as the target

## Import Modules

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from ast import literal_eval
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import json
import csv

## Load the data of first stage decisions, uncertainities, instances and second stage objectives

In [2]:
pwd()

'/ztank/scratch/user/u.rd143338/ss_from_nn/Neural_second_stage/post_train_instance_20'

In [3]:
## Directory of data for 10 items
I = 20
scen = 50
direc = '/ztank/scratch/user/u.rd143338/ss_from_nn'
filename = "instance_1_250_items_" + str(I) + "_num_of_first_stage_11_scenarios_" + str(scen)

# path contains the location to the csv files
path=os.path.join(direc, filename)

In [4]:
## folder to save the data

def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")
        
##################################################################################        
##################################################################################
# Specify the folder path you want to create
arch_date_created = "high layers improved NN"


##################################################################################
##################################################################################
#### this folder path is for data storage
folder_path = os.path.join(arch_date_created)

# Call the function to create the folder
create_folder(folder_path)

Folder 'high layers improved NN' already exists.


In [5]:
def save_weights_biases_to_json(model, filename):
    # Extract weights and biases from the model
    weights_biases = {}
    for name, param in model.named_parameters():
        weights_biases[name] = param.data.tolist()

    # Save weights and biases to a JSON file
    with open(filename, 'w') as json_file:
        json.dump(weights_biases, json_file)

In [6]:
print("Input file path is at: {}".format(folder_path))

Input file path is at: high layers improved NN


In [7]:
folder_path

'high layers improved NN'

In [8]:
def combine_csv_data(folder_path):
    # Get a list of all CSV files in the folder
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    
    # Initialize an empty DataFrame to store combined data
    dfs = []
    
    # Loop through each CSV file
    for file in csv_files:
        # Read the CSV file into a DataFrame
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        
        # Append the DataFrame to the combined DataFrame
        dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

## Combined data from csv files
combined_data = combine_csv_data(path)

In [9]:
combined_data.head(2)

,f,first_stage_obj,x,r,second_stage_obj,p_bar,Reduced Capacity,t,p_hat,total_obj,y,original_capacity,w,uncern,gamma,seed
0,"[786.7367467600798, 43.66934851926846, 162.193...",1926.467568,"[-0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, 1.0,...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -0.0, 1.0,...",-7550.168084,"[659.1369113656739, 30.554869499429, 113.97742...",2934.110215,"[295.2501409369704, 424.6079924086646, 103.184...","[298.010362187704, 16.585270654569555, 62.4610...",-5623.700516,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...",3912.146954,"[570.4489230680714, 561.4783723073962, 391.224...","[0.0709, 0.251, 0.0441, 1.0184, 0.1205, 0.3195...",4.0,32
1,"[786.7367467600798, 43.66934851926846, 162.193...",1926.467568,"[-0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, 1.0,...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...",-7366.064879,"[659.1369113656739, 30.554869499429, 113.97742...",2934.110215,"[295.2501409369704, 424.6079924086646, 103.184...","[298.010362187704, 16.585270654569555, 62.4610...",-5439.597312,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...",3912.146954,"[570.4489230680714, 561.4783723073962, 391.224...","[0.0492, 0.1748, 0.1158, 0.153, 0.0711, 0.0452...",4.0,32


## Shuffle the dataset for training

In [10]:
# Use only seed from 1 to 200 for training
combined_data = combined_data[combined_data["seed"]<=200]
shuffle_seed = 1
training_data=combined_data.sample(frac=1, random_state=shuffle_seed).reset_index(drop=True)
training_data.head(2)

,f,first_stage_obj,x,r,second_stage_obj,p_bar,Reduced Capacity,t,p_hat,total_obj,y,original_capacity,w,uncern,gamma,seed
0,"[668.3026011818135, 1287.8562336053396, 538.67...",2407.90594,"[1.0, 0.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0...","[1.0, 0.0, 0.0, 1.0, 0.0, -0.0, -0.0, 0.0, 0.0...",-9058.417905,"[527.5306212418371, 923.419571417487, 401.5972...",2840.612347,"[36.299713207101334, 229.23908585200888, 142.2...","[256.3094599686253, 442.5731412958027, 207.114...",-6650.511965,"[1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",3341.896879,"[301.98683454640326, 768.1277137348278, 485.34...","[0.0272, 0.0967, 0.5364, 0.0202, 0.0663, 0.039...",2.0,70
1,"[703.0926362237493, 1186.4954312719087, 283.71...",1862.06151,"[-0.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, ...","[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-9037.251754,"[561.4235703171213, 913.832777448831, 219.6439...",3073.338103,"[642.5177086820069, 151.71523729802115, 224.39...","[263.86371393025183, 438.9654149836514, 99.269...",-7175.190244,"[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",3414.820114,"[942.9299427166219, 624.6242716623353, 296.082...","[0.1447, 0.4878, 0.2543, 0.1594, 0.2792, 0.006...",4.0,192


## The element list of the dataframe is split and new dataframe for instances, X and uncertainities are formed

In [11]:
## The cells have list, make one column for each member of the list
def split_list_column(df, col_name):
    col=df[col_name].apply(literal_eval).apply(pd.Series)
    new_columns = [f'{col_name}_{i}' for i in range(1,len(col.columns)+1)]
    col.columns=new_columns
    return col

In [12]:
# Instance parameter that define a knapsack problem, these words will be used as a prefic for the columns generated by the elements of the list
instance_parameter = ['f', 'p_bar', 't', 'p_hat', 'original_capacity', 'w', 'gamma']

In [13]:
instance_df=pd.DataFrame()
for i in instance_parameter:
    if i!='original_capacity' and i!='gamma':
        instance_df=pd.concat([instance_df, split_list_column(training_data,i)], axis=1)
    else:
        instance_df=pd.concat([instance_df, training_data[i]], axis=1)

In [14]:
instance_df.shape

(110000, 102)

In [15]:
p_bar = instance_df.loc[: , instance_df.columns.str.startswith('p_bar')].reset_index(drop=True)
p_bar.head(2)

,p_bar_1,p_bar_2,p_bar_3,p_bar_4,p_bar_5,p_bar_6,p_bar_7,p_bar_8,p_bar_9,p_bar_10,p_bar_11,p_bar_12,p_bar_13,p_bar_14,p_bar_15,p_bar_16,p_bar_17,p_bar_18,p_bar_19,p_bar_20
0,527.530621,923.419571,401.597263,601.986693,973.604434,281.057600,746.491479,192.692532,439.342751,583.819419,94.037998,545.867703,439.227213,935.126780,200.056757,404.037751,898.655196,895.128889,429.041162,832.264044
1,561.423570,913.832777,219.643966,475.713350,193.920942,50.703086,870.473152,169.655321,744.802975,672.477532,710.186987,206.553695,889.163072,965.945607,118.752765,805.729976,63.063472,980.607714,490.019986,690.109213


In [16]:
f = instance_df.loc[: , instance_df.columns.str.startswith('f')].reset_index(drop=True)
f.head(2)

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20
0,668.302601,1287.856234,538.675378,851.961267,1206.810584,317.726331,1028.294226,251.171090,511.985739,719.841161,113.234803,634.827844,637.323233,1133.082635,237.480672,480.385927,1336.758191,1299.972628,553.383710,1188.579139
1,703.092636,1186.495431,283.715212,611.909800,224.091454,68.136526,1176.598004,238.010931,997.537363,854.939961,1019.436797,269.306064,1015.967665,1244.503420,164.074143,1086.553392,74.766895,1078.941370,585.277262,894.657645


In [17]:
X_df=split_list_column(training_data, "x")
X_df.head(2)

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,x_20
0,1.0,0.0,-0.0,1.0,-0.0,1.0,1.0,-0.0,-0.0,1.0,-0.0,1.0,1.0,-0.0,-0.0,1.0,1.0,1.0,-0.0,1.0
1,-0.0,1.0,-0.0,1.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,1.0,1.0,1.0,-0.0,1.0,-0.0,1.0,1.0,1.0


In [18]:
Uncer_df=split_list_column(training_data, "uncern")
Uncer_df.head(2)

,uncern_1,uncern_2,uncern_3,uncern_4,uncern_5,uncern_6,uncern_7,uncern_8,uncern_9,uncern_10,uncern_11,uncern_12,uncern_13,uncern_14,uncern_15,uncern_16,uncern_17,uncern_18,uncern_19,uncern_20
0,0.0272,0.0967,0.5364,0.0202,0.0663,0.0391,0.0785,0.0272,0.1290,0.0228,0.0894,0.0080,0.0623,0.0703,0.0384,0.2126,0.2223,0.0617,0.0655,0.0531
1,0.1447,0.4878,0.2543,0.1594,0.2792,0.0064,0.0691,0.6022,0.0814,0.0945,0.4163,0.2567,0.3548,0.0313,0.0237,0.0322,0.3127,0.0898,0.1163,0.0024


In [19]:
target=training_data["total_obj"]
target

0         -6650.511965
1         -7175.190244
2         -6853.433127
3         -4872.761981
4        -11154.401175
              ...     
109995    -8932.247559
109996    -8977.719958
109997    -8084.886084
109998    -7960.723633
109999   -10088.266736
Name: total_obj, Length: 110000, dtype: float64

## Neural Network Training

In [20]:
# Convert target column to numpy array
target_array = target.values
target_array

array([ -6650.51196482,  -7175.19024388,  -6853.43312693, ...,
        -8084.88608393,  -7960.72363269, -10088.2667364 ])

In [21]:
## For instance = 20
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        ## Dont change these values
        # self.embedding_instance = nn.Linear(102, 128)
        # self.embedding_X = nn.Linear(20, 15)
        # self.embedding_uncern = nn.Linear(20, 15)
        # self.fc1 = nn.Linear(158, 54)
        # self.fc2 = nn.Linear(54, 32)  
        # self.fc3 = nn.Linear(32, 8)
        # self.fc4 = nn.Linear(8,1)
        
        self.embedding_instance = nn.Linear(102, 228)
        self.embedding_X = nn.Linear(20, 30)
        self.embedding_uncern = nn.Linear(20, 30)
        self.fc1 = nn.Linear(288, 256)
        self.fc2 = nn.Linear(256, 128)  
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 10)
        self.fc6 = nn.Linear(10,1)


    def forward(self, instance, X, uncern):
        # instance_embed = torch.relu(self.embedding_instance(instance))
        # X_embed = torch.relu(self.embedding_X(X))
        # uncern_embed = torch.relu(self.embedding_uncern(uncern))
        # concatenated = torch.cat((instance_embed, X_embed, uncern_embed), dim=1)
        
        concatenated = torch.cat((self.embedding_instance(instance),self.embedding_X(X), self.embedding_uncern(uncern)), dim=1)
        
        output = torch.relu(self.fc1(concatenated))
        output = torch.relu(self.fc2(output))
        output = torch.relu(self.fc3(output))
        output = torch.relu(self.fc4(output))
        output = torch.relu(self.fc5(output))
        output = self.fc6(output)
        return output

In [22]:
# Split data into training and test sets
instance_train, instance_test, \
X_train, X_test, \
uncern_train, uncern_test, \
target_train, target_test = train_test_split(instance_df.values, X_df.values, Uncer_df.values, target_array, test_size=0.2, random_state=1)

### Since the X_train is always scaled between 0 or 1, here we shall just scale the values of the intsnace_train and uncern_train. Once this is achieved, we will get the values of x_min and x_max from the training set which we wil further use to scale the values of the testing set.


In [23]:
scaler_instance = MinMaxScaler()
scaler_instance.fit(instance_train)
instance_train_transformed = scaler_instance.transform(instance_train)
instance_test_transformed = scaler_instance.transform(instance_test)

In [24]:
scaler_uncern = MinMaxScaler()
scaler_uncern.fit(uncern_train)
uncern_train_transformed = scaler_uncern.transform(uncern_train)
uncern_test_transformed = scaler_uncern.transform(uncern_test)

In [25]:
scaler_target = MinMaxScaler()
scaler_target.fit(target_train.reshape(-1,1))
target_train_transformed = scaler_target.transform(target_train.reshape(-1,1))
target_test_transformed = scaler_target.transform(target_test.reshape(-1,1))

In [26]:
min_max_scalers={}
min_max_scalers["scaler_instance.data_max_"]=scaler_instance.data_max_.tolist()
min_max_scalers["scaler_instance.data_min_"]=scaler_instance.data_min_.tolist()
min_max_scalers["scaler_uncern.data_max_"]=scaler_uncern.data_max_.tolist()
min_max_scalers["scaler_uncern.data_min_"]=scaler_uncern.data_min_.tolist()
min_max_scalers["scaler_target.data_max_"]=scaler_target.data_max_.tolist()
min_max_scalers["scaler_target.data_min_"]=scaler_target.data_min_.tolist()

file_name = "min_max_scalers_inst_" + str(I) + ".json"
file_path = os.path.join(folder_path, file_name)

# Write dictionary to JSON file
with open(file_path, 'w') as json_file:
    json.dump(min_max_scalers, json_file)

print("JSON file created successfully!")

JSON file created successfully!


In [27]:
pwd()

'/ztank/scratch/user/u.rd143338/ss_from_nn/Neural_second_stage/post_train_instance_20'

In [28]:
folder_path

'high layers improved NN'

In [29]:
# Convert numpy arrays to PyTorch tensors
instance_train_tensor = torch.tensor(instance_train_transformed, dtype=torch.float32)
instance_test_tensor = torch.tensor(instance_test_transformed, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
uncern_train_tensor = torch.tensor(uncern_train_transformed, dtype=torch.float32)
uncern_test_tensor = torch.tensor(uncern_test_transformed, dtype=torch.float32)
target_train_tensor = torch.tensor(target_train_transformed, dtype=torch.float32)
target_test_tensor = torch.tensor(target_test_transformed, dtype=torch.float32)

In [30]:
# Create training and test datasets
train_dataset = TensorDataset(instance_train_tensor, X_train_tensor, uncern_train_tensor, target_train_tensor)
test_dataset = TensorDataset(instance_test_tensor, X_test_tensor, uncern_test_tensor, target_test_tensor)


In [31]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [32]:
# Move model to GPU
model = MyModel().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
# Create training and test data loaders
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [35]:
optimizer = optim.Adamax(model.parameters(), lr=0.01)

# Define learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=50, verbose=True)

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    running_loss = 0.0
    running_mae = 0.0  # Initialize running MAE
    for i, data in enumerate(train_loader):
        inputs_instance, inputs_X, inputs_uncern, labels = data
        # Move tensors to GPU
        inputs_instance, inputs_X, inputs_uncern, labels = inputs_instance.to(device), inputs_X.to(device), inputs_uncern.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs_instance, inputs_X, inputs_uncern)
        loss = criterion(outputs, labels.view(-1, 1))  # Assuming labels is a column vector
        
        # Calculate Mean Absolute Error (MAE)
        mae = torch.mean(torch.abs(outputs - labels.view(-1, 1)))
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        running_mae += mae.item()  # Accumulate MAE
        
        ######## Print the true loss on the inverse transformed variables
        
        ############################################################################
        outputs_invtransformed = scaler_target.inverse_transform(outputs.cpu().detach().numpy())
        labels_invtransformed = scaler_target.inverse_transform(labels.view(-1, 1).cpu().detach().numpy())
        outputs_invtransformed = outputs_invtransformed.flatten()
        labels_invtransformed = labels_invtransformed.flatten()
        # Calculate MSE
        mae_invtransformed = np.mean(np.abs(outputs_invtransformed - labels_invtransformed))
        # Calculate MAPE
        mape_invtransformed = np.mean(np.abs((outputs_invtransformed - labels_invtransformed) /labels_invtransformed)) * 100
        #############################################################################
        
        
        if epoch % 100 == 0 and (i + 1) % 100 == 0:  # Print every 100 epochs and 100th mini-batch
            print('[%d, %5d] epoch_loss: %.6f batch_loss: %.6f batch_mae: %.6f' %
                  (epoch + 1, i + 1, running_loss / 100, loss.item(), mae.item()))
            running_loss = 0.0
            
            print('True MAE: %.6f , True MAPE: %.6f '%
                 (mae_invtransformed, mape_invtransformed))
    
        ############################################################################
    
    # Perform validation and adjust learning rate
    with torch.no_grad():
        test_loss = 0.0
        test_mae = 0.0
        mae_invtransformed_test = 0.0
        mape_invtransformed_test = 0.0
        for data in test_loader:
            inputs_instance, inputs_X, inputs_uncern, labels = data
            # Move tensors to GPU
            inputs_instance, inputs_X, inputs_uncern, labels = inputs_instance.to(device), inputs_X.to(device), inputs_uncern.to(device), labels.to(device)
            
            outputs = model(inputs_instance, inputs_X, inputs_uncern)
            test_loss += criterion(outputs, labels.view(-1, 1)).item()
            test_mae += torch.mean(torch.abs(outputs - labels.view(-1, 1))).item()  # Calculate MAE for test set
            
            ####################################################################################
            outputs_invtransformed_test = scaler_target.inverse_transform(outputs.cpu().detach().numpy())
            labels_invtransformed_test = scaler_target.inverse_transform(labels.view(-1, 1).cpu().detach().numpy())
            outputs_invtransformed_test = outputs_invtransformed.flatten()
            labels_invtransformed_test = labels_invtransformed.flatten()
            # Calculate MSE
            mae_invtransformed_test += np.mean(np.abs(outputs_invtransformed_test - labels_invtransformed_test))
            # Calculate MAPE
            mape_invtransformed_test += np.mean(np.abs((outputs_invtransformed_test - labels_invtransformed_test) /labels_invtransformed_test)) * 100
            
            ####################################################################################
            
        scheduler.step(test_loss)
        
    if epoch % 100 == 0:  # Print every 100 epochs
        print('[%d] Training MAE: %.6f ***Validation*** MAE: %.6f #######  True MAE on test batch %.3f , True MAPE on test batch %.3f' % (epoch + 1, running_mae / len(train_loader), test_mae / len(test_loader), mae_invtransformed_test/len(test_loader), mape_invtransformed_test/len(test_loader)))

print('Finished Training')

[1,   100] epoch_loss: 0.000004 batch_loss: 0.000003 batch_mae: 0.001272
True MAE: 11.941164 , True MAPE: 0.160084 
[1] Training MAE: 0.001387 ***Validation*** MAE: 0.002969 #######  True MAE on test batch 10.670 , True MAPE on test batch 0.142
Epoch 00032: reducing learning rate of group 0 to 9.0000e-04.
Epoch 00063: reducing learning rate of group 0 to 8.1000e-04.
Epoch 00094: reducing learning rate of group 0 to 7.2900e-04.
[101,   100] epoch_loss: 0.000002 batch_loss: 0.000003 batch_mae: 0.001406
True MAE: 13.197083 , True MAPE: 0.187167 
[101] Training MAE: 0.001156 ***Validation*** MAE: 0.002941 #######  True MAE on test batch 9.662 , True MAPE on test batch 0.135
Epoch 00125: reducing learning rate of group 0 to 6.5610e-04.
Epoch 00156: reducing learning rate of group 0 to 5.9049e-04.
Epoch 00187: reducing learning rate of group 0 to 5.3144e-04.
[201,   100] epoch_loss: 0.000002 batch_loss: 0.000003 batch_mae: 0.001280
True MAE: 12.018722 , True MAPE: 0.170851 
[201] Training MA

In [37]:
# Specify the file path where you want to save the model weights

weights_path = "model_weights_" + str(I) + "_instance_" + str(scen) + "_scen.pth"
file_path =os.path.join(folder_path, weights_path)
# Save the model state dictionary
torch.save(model.state_dict(), file_path)

print("Model weights saved successfully at:", file_path)

Model weights saved successfully at: high layers improved NN/model_weights_20_instance_50_scen.pth


In [38]:
file_name = "model_weight_" + str(I) + "_instance_" + str(scen) + "_scen.json"
file_path = os.path.join(folder_path, file_name)
save_weights_biases_to_json(model, file_path)

In [39]:
# Create a list to store dictionaries of data
data_dicts = []

problem_size = instance_df.shape[1]


# Ensure model and data are on the same device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Move model to the selected device
model = model.to(device)

# Iterate through test loader
with torch.no_grad():
    for data in test_loader:
        inputs_instance, inputs_X, inputs_uncern, labels = data
        inputs_instance = inputs_instance.to(device)
        inputs_X = inputs_X.to(device)
        inputs_uncern = inputs_uncern.to(device)
        labels = labels.to(device)

        # Forward pass through the model
        outputs = model(inputs_instance, inputs_X, inputs_uncern)

        # Convert outputs and labels to numpy arrays (if needed)
        outputs = outputs.cpu().numpy()
        labels = labels.cpu().numpy()
        # Iterate through batch
        for i in range(len(labels)):
            # Prepare a dictionary for row data
            row_data = {}
            for j in range(problem_size):
                row_data['Input_Instance_' + str(j)] = inputs_instance[i][j].item()
            for j in range(I):
                row_data['Input_X_' + str(j)] = inputs_X[i][j].item()
                row_data['Input_Uncern_' + str(j)] = inputs_uncern[i][j].item()
            row_data['Predicted_Output'] = outputs[i][0]
            row_data['Real_Output'] = labels[i]
            
            # Append row dictionary to the list
            data_dicts.append(row_data)

# Create DataFrame from the list of dictionaries
test_data_df = pd.DataFrame(data_dicts)
test_data_df = pd.concat([test_data_df.iloc[:,:problem_size], test_data_df[sorted(test_data_df.iloc[:,problem_size:])]], axis=1)
# # Save DataFrame to a CSV file
# data_df.to_csv('test_results_scaled_mini.csv', index=False)

# print("CSV file saved successfully.")


In [40]:
test_data_df.shape

(22000, 144)

In [41]:
# Create a list to store dictionaries of data
data_dicts = []

problem_size = instance_df.shape[1]


# Ensure model and data are on the same device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Move model to the selected device
model = model.to(device)

# Iterate through test loader
with torch.no_grad():
    for data in train_loader:
        inputs_instance, inputs_X, inputs_uncern, labels = data
        inputs_instance = inputs_instance.to(device)
        inputs_X = inputs_X.to(device)
        inputs_uncern = inputs_uncern.to(device)
        labels = labels.to(device)

        # Forward pass through the model
        outputs = model(inputs_instance, inputs_X, inputs_uncern)

        # Convert outputs and labels to numpy arrays (if needed)
        outputs = outputs.cpu().numpy()
        labels = labels.cpu().numpy()
        # Iterate through batch
        for i in range(len(labels)):
            # Prepare a dictionary for row data
            row_data = {}
            for j in range(problem_size):
                row_data['Input_Instance_' + str(j)] = inputs_instance[i][j].item()
            for j in range(I):
                row_data['Input_X_' + str(j)] = inputs_X[i][j].item()
                row_data['Input_Uncern_' + str(j)] = inputs_uncern[i][j].item()
            row_data['Predicted_Output'] = outputs[i][0]
            row_data['Real_Output'] = labels[i]
            
            # Append row dictionary to the list
            data_dicts.append(row_data)

# Create DataFrame from the list of dictionaries
train_data_df = pd.DataFrame(data_dicts)
train_data_df = pd.concat([train_data_df.iloc[:,:problem_size], train_data_df[sorted(train_data_df.iloc[:,problem_size:])]], axis=1)
# # Save DataFrame to a CSV file
# data_df.to_csv('test_results_scaled_mini.csv', index=False)

# print("CSV file saved successfully.")


In [42]:
train_data_df

,Input_Instance_0,Input_Instance_1,Input_Instance_2,Input_Instance_3,Input_Instance_4,Input_Instance_5,Input_Instance_6,Input_Instance_7,Input_Instance_8,Input_Instance_9,...,Input_X_2,Input_X_3,Input_X_4,Input_X_5,Input_X_6,Input_X_7,Input_X_8,Input_X_9,Predicted_Output,Real_Output
0,0.522343,0.840937,0.273446,0.226863,0.534307,0.872936,0.803643,0.404864,0.468557,0.515224,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.344252,[0.34368682]
1,0.667587,0.603345,0.615207,0.321750,0.679766,0.622413,0.608257,0.772115,1.000000,0.526591,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.297345,[0.29774976]
2,0.852066,0.035834,0.369676,0.589228,0.493904,0.647270,0.130130,0.480853,0.683585,0.780598,...,0.0,1.0,0.0,0.0,-0.0,-0.0,1.0,1.0,0.569108,[0.5690927]
3,0.046207,0.077465,0.490825,0.250275,0.693002,0.834935,0.764190,0.053456,0.086031,0.836928,...,1.0,1.0,1.0,1.0,1.0,-0.0,1.0,1.0,0.554132,[0.55609274]
4,0.590604,0.492968,0.100451,0.753251,0.641162,0.036551,0.597009,0.882112,0.727744,0.055532,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.335619,[0.33641803]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87995,0.545245,0.628936,0.092196,0.659909,0.493366,0.635638,0.098929,0.612639,0.257343,0.633411,...,-0.0,-0.0,1.0,1.0,-0.0,-0.0,-0.0,1.0,0.819742,[0.81865495]
87996,0.820210,0.023222,0.382725,0.800134,0.682976,0.345339,0.909329,0.388448,0.844921,0.244721,...,-0.0,-0.0,1.0,-0.0,1.0,-0.0,1.0,-0.0,0.714784,[0.71775025]
87997,0.308201,0.376821,0.125786,0.407443,0.029105,0.938814,0.043374,0.915889,0.667858,0.726472,...,-0.0,1.0,-0.0,1.0,-0.0,1.0,1.0,1.0,0.483473,[0.48384356]
87998,0.485027,0.854712,0.197442,0.440022,0.153848,0.041720,0.819249,0.161502,0.704876,0.574145,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.688540,[0.688617]


In [43]:
data_df = pd.concat([train_data_df, test_data_df])
file_name = 'post_NN_results_' + str(I) + '_instance_'+ str(scen) +'_scen.csv'
file_path =os.path.join(folder_path, file_name)
data_df.to_csv(file_path, index=False)

print("CSV file saved successfully.")

CSV file saved successfully.


In [44]:
# Function to hook to the forward pass of all layers
def activation_hook(name):
    def hook(module, input, output):
        activated_neurons = torch.relu(output) > 0  # Using ReLU activation function
        activations_dict[name] = activated_neurons
    return hook

In [45]:
folder_path

'high layers improved NN'

In [73]:
# I=10
# row = 50
#     # Create an instance of your model
# model = MyModel()
# file_name = "model_weights_" + str(I) + ".pth"
# file_path = os.path.join(folder_path, file_name)

# model.load_state_dict(torch.load(file_path))

# # Dictionary to store activated neurons for each layer
# activations_dict = {}

# # Register the hook to all layers of the model
# for name, module in model.named_children():
#     module.register_forward_hook(activation_hook(name))

# with torch.no_grad():
#     output = model(train_dataset.tensors[0][row].reshape(1,-1), train_dataset.tensors[1][row].reshape(1,-1), train_dataset.tensors[2][row].reshape(1,-1))
# result = np.concatenate([activations_dict[key].numpy().reshape(-1) for key in ['fc1', 'fc2', 'fc3','fc4']])
# # Print the activated neurons for each layer
# for layer_name, activated_neurons in activations_dict.items():
#     print(f"Activated neurons for {layer_name}: {activated_neurons}")

In [74]:
# result = []
# n_row = train_dataset.tensors[0].shape[0]
# for i in range(0,n_row):
#     model = MyModel()
#     model.load_state_dict(torch.load(file_path))

#     # Dictionary to store activated neurons for each layer
#     activations_dict = {}

#     # Register the hook to all layers of the model
#     for name, module in model.named_children():
#         module.register_forward_hook(activation_hook(name))

#     with torch.no_grad():
#         output = model(train_dataset.tensors[0][i].reshape(1,-1), train_dataset.tensors[1][i].reshape(1,-1), train_dataset.tensors[2][i].reshape(1,-1))
#     result.append(np.concatenate([activations_dict[key].numpy().reshape(-1) for key in ['fc1', 'fc2', 'fc3', 'fc4']]))
# activations = pd.DataFrame(result)

In [74]:
# result_test = []
# n_row_t = test_dataset.tensors[0].shape[0]
# for i in range(0,n_row_t):
#     model = MyModel()
#     model.load_state_dict(torch.load(file_path))

#     # Dictionary to store activated neurons for each layer
#     activations_dict = {}

#     # Register the hook to all layers of the model
#     for name, module in model.named_children():
#         module.register_forward_hook(activation_hook(name))

#     with torch.no_grad():
#         output = model(test_dataset.tensors[0][i].reshape(1,-1), test_dataset.tensors[1][i].reshape(1,-1), test_dataset.tensors[2][i].reshape(1,-1))
#     result_test.append(np.concatenate([activations_dict[key].numpy().reshape(-1) for key in ['fc1', 'fc2', 'fc3']]))

# activations_test = pd.DataFrame(result_test)


In [75]:
# act_train = (activations.sum()/train_dataset.tensors[0].shape[0]).to_list()
# act_test = (activations_test.sum()/test_dataset.tensors[0].shape[0]).to_list()
# # for i in range(len(activations)):
# #     print(f"Train: {round(act_train[i],4)},  Test: {round(act_test[i],4)}")

# for i in range(len(act_train)):
#     train_activation = act_train[i]
#     test_activation = act_test[i]

#     train_text = f"Train: {round(train_activation, 4)}"
#     test_text = f"Test: {round(test_activation, 4)}"

#     # Check if activations are 0 and change color accordingly
#     if train_activation == 0:
#         train_text = f"\033[91m{train_text}\033[0m"  # Red color
#     if test_activation == 0:
#         test_text = f"\033[91m{test_text}\033[0m"  # Red color

#     print(train_text, test_text)